# On Matplotlib

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import matplotlib.animation as animation
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
def generate_points_data(num_points, scale):
    # Generate new random points scaled as specified
    return np.random.rand(num_points, 3) * scale
    
# Example: Generating datasets separately
num_frames = 100
num_points = 100
scale = 2

# For 1 object.
#data_points = generate_points_data(num_points, scale)
#datasets = [data_points for _ in range(num_frames)]

datasets = [generate_points_data(num_points, scale) for _ in range(num_frames)]

In [3]:
class Faces():
    def __init__(self,tri, sig_dig=12, method="convexhull"):
        self.method=method
        self.tri = np.around(np.array(tri), sig_dig)
        self.grpinx = list(range(len(tri)))
        norms = np.around([self.norm(s) for s in self.tri], sig_dig)
        _, self.inv = np.unique(norms,return_inverse=True, axis=0)

    def norm(self,sq):
        cr = np.cross(sq[2]-sq[0],sq[1]-sq[0])
        return np.abs(cr/np.linalg.norm(cr))

    def isneighbor(self, tr1,tr2):
        a = np.concatenate((tr1,tr2), axis=0)
        return len(a) == len(np.unique(a, axis=0))+2

    def order(self, v):
        if len(v) <= 3:
            return v
        v = np.unique(v, axis=0)
        n = self.norm(v[:3])
        y = np.cross(n,v[1]-v[0])
        y = y/np.linalg.norm(y)
        c = np.dot(v, np.c_[v[1]-v[0],y])
        if self.method == "convexhull":
            h = ConvexHull(c)
            return v[h.vertices]
        else:
            mean = np.mean(c,axis=0)
            d = c-mean
            s = np.arctan2(d[:,0], d[:,1])
            return v[np.argsort(s)]

    def simplify(self):
        for i, tri1 in enumerate(self.tri):
            for j,tri2 in enumerate(self.tri):
                if j > i: 
                    if self.isneighbor(tri1,tri2) and \
                       self.inv[i]==self.inv[j]:
                        self.grpinx[j] = self.grpinx[i]
        groups = []
        for i in np.unique(self.grpinx):
            u = self.tri[self.grpinx == i]
            u = np.concatenate([d for d in u])
            u = self.order(u)
            groups.append(u)
        return groups



def calculate_camera_angles(num, rotate_axes, rotate_angle, camera_position):
    # Calculate base elev and azim from camera position
    x, y, z = camera_position
    r = math.sqrt(x**2 + y**2 + z**2)
    elev = math.degrees(math.asin(z / r))  # Base elevation angle
    azim = math.degrees(math.atan2(y, x))  # Base azimuthal angle

    # Apply rotation on top of base angles for each specified axis
    for axis in rotate_axes:
        if axis == 'x':
            elev += rotate_angle * num
        elif axis == 'y':
            azim += rotate_angle * num
        elif axis == 'z':
            elev += rotate_angle * num
            azim += rotate_angle * num

    return elev, azim

def plot_scatter(ax, points, alpha):
    # Create convex hull
    hull = ConvexHull(points)
    simplices = hull.simplices

    # Original triangles from the convex hull
    org_triangles = [points[s] for s in simplices]

    # Process faces
    f = Faces(org_triangles)
    g = f.simplify()

    # Assuming g is your list of faces for the Poly3DCollection
    facecolors = np.array(['green' for _ in g])
    edgecolors = np.array(['black' for _ in g])
    
    # Plot the faces
    pc = Poly3DCollection(g, 
                          facecolors=facecolors, 
                          edgecolors=edgecolors,
                          linewidths=1.5, 
                          alpha=alpha, 
                          zorder=-3, 
                          shade=True
                         )
    ax.add_collection3d(pc)

    # Plot all points - non-vertices in blue
    for i in range(len(points)):
        if i not in hull.vertices:
            ax.scatter(points[i, 0], points[i, 1], points[i, 2], color='blue', s=10, zorder=3)

    # Plot vertices (corners) of the convex hull in red with larger size
    for vertex in hull.vertices:
        ax.scatter(points[vertex, 0], points[vertex, 1], points[vertex, 2], color='red', s=50, zorder=4)

    return ax


def update(num, ax, fig, datasets, rotate_axes, rotate_angle, camera_position, alpha):
    ax.clear()  # Clear current plot

    # Setting labels
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_title(f"Frame {num}")

    # Calculate camera angles and set camera view
    elev, azim = calculate_camera_angles(num, rotate_axes, rotate_angle, camera_position)
    ax.view_init(elev=elev, azim=azim)

    # Plot scatter points with transparency and return updated ax
    points = datasets[num]
    ax = plot_scatter(ax, points, alpha)

    return fig,

def run_animation(datasets, rotate_axes=['z'], rotate_angle=1, camera_position=(1, 1, 1), alpha=0.5, interval=100):
    # Create figure and axes
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Create animation with specified settings
    ani = animation.FuncAnimation(fig, update, frames=len(datasets), 
                                  fargs=(ax, fig, datasets, rotate_axes, rotate_angle, camera_position, alpha), 
                                  interval=interval)

    # Return the animation
    return ani

In [ ]:
# Example: Running the animation with the pre-generated datasets and specified transparency
ani = run_animation(datasets, rotate_axes=['x', 'y'], rotate_angle=3, camera_position=(1, 1, 1), alpha=1, interval=150)

In [5]:
from IPython.display import HTML
HTML(ani.to_html5_video())